<a href="https://colab.research.google.com/github/sgomiz/GEE/blob/master/LST_2_Dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example for getting LST values in a pandas data frame by means of a MODIS satellite image collection

---
**Author:** Sergio Gómiz Morcillo
**Versión:** 0.1
**Date:** July 5, 2019

**Description:**

Following this example we can obtain the LST (Land Surface Temperature) values for a specific data range and region of interest. These data series come from a collection of images from the MODIS satellite (**MODIS/006/MOD11A2**).
To get this collection of images. It is necessary to have previously an active GEE account.

This example use both geometry possibilites: polygon or point (coordinates) and it should be like the region parameter to the **LST2DF function.**

Considerations to bear in mind: GEE API have strengh limitations about size of result dataset (getRegion function) . Raised error . **Too many values**: 42622 points x 10 bands x 685 images > 1048576










In [0]:
# Show ROI over the map
from IPython.display import Image
Image(filename="ROI_Cadiz.PNG")

# Prerrequisite: Google Earth Engine API

In [0]:
# Installing GEE API
!pip install earthengine-api

In [0]:
# Getting the access token to GEE API
!earthengine authenticate

# Function LST2DF
Own function to encapsulate all core coding, in order to get to MODIS images collection (**MODIS/006/MOD11A2**) and to get the LST data series.

In [0]:
def  LST2DF(coordinates, region, date_start, end_date):
  #Get the LST (Land Surface Temperature) from MODIS Satellite via Google Earth Engine
  #coordinates : array of coordinates long/lat
  #region : polygon or point values
  #date_start : Starting Date for setup date range
  #end_date : Ending Date for setup date range 
 
  # Define the type of geometry for setting properly the region coordinates
  if region == 'polygon' :
      ROI = ee.Geometry.Polygon(coordinates)
  elif region == 'point' :  
      ROI = ee.Geometry.Point(coordinates)
  else:
      raise KeyError('Wrong region value, allowed values are polygon or point')

  # Define the image (scala 1000)
  img = ee.ImageCollection('MODIS/006/MOD11A2').filterDate(date_start, end_date)
  result = img.select('LST_Day_1km').getRegion(ROI,1000).getInfo()

  # Export to dataframe
  df_LST = pd.DataFrame(result[1:])
  df_LST.columns = result[0]
  
  # Switching LST from Kelvin to Celsius
  df_LST['LST_Celsius'] = df_LST.groupby(['id']).transform(sum)['LST_Day_1km'] * 0.02-273.15
   
  
  return df_LST


# Testing Function Code
Coordinates define a Region Of Interest (ROI) to collect some data series from a specific region over the map.

In [0]:
# Main Program Code to call function NDVI2DF

# Gettting Date Range
date_start = ee.Date('2019-03-01')
end_date = ee.Date('2019-06-30')

# Setting type of geometry for region of interest (point or polygon)
region = 'point'

# Getting ROI through array of coordinates (point of polygon)
coordinates = [6.134136, 49.612485]
#coordinates = [[-6.151065585896504, 36.29423686205435],
#          [-6.035709140584004, 36.19012306037972],
#          [-5.601749179646504, 36.00370222379312],
#          [-5.313358066365254, 36.21228654591974],
#          [-5.516605136677754, 36.87866023265441],
#          [-6.428470371052754, 36.73131853399765]]

df_LST = LST2DF (coordinates, region, date_start, end_date)
df_LST

,id,longitude,latitude,time,LST_Day_1km,LST_Celsius
0,2019_03_06,6.131002,49.609462,1551830400000,14186,10.57
1,2019_03_14,6.131002,49.609462,1552521600000,14512,17.09
2,2019_03_22,6.131002,49.609462,1553212800000,14500,16.85
3,2019_03_30,6.131002,49.609462,1553904000000,14697,20.79
4,2019_04_07,6.131002,49.609462,1554595200000,14547,17.79
5,2019_04_15,6.131002,49.609462,1555286400000,14957,25.99
6,2019_04_23,6.131002,49.609462,1555977600000,14835,23.55
7,2019_05_01,6.131002,49.609462,1556668800000,14993,26.71
8,2019_05_09,6.131002,49.609462,1557360000000,14728,21.41
9,2019_05_17,6.131002,49.609462,1558051200000,15177,30.39
